<h1>Segmentation and clustering fo the Neighbourhood of the Canda</h1>

<h2>Introduction:</h2>
Aim of this project is to  analyz  the neighbourhood of the Canda.Here we will use wiki web page then we scraping by using python webscraping packages beautifulsoup ,then we put this tuple text data into our dataframe.
<li><a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">wiki link....</a></li>

<h3>1. Import the packages</h3>

In [1]:
#Import Beautiful Soup, lxml, requests to scrap data from Toronto Neighborhood in Wikipedia
!pip install wikipedia
import wikipedia as wp
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # to rendering the html DOM
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("Libraries Imported!")


  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11686 sha256=25f2b994575f525080fbd04b1186a7f30744a6a149953b428d52114526a07eca
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Libraries Imported!


<h2>2. Web scraping for data collection</h2>

<p>2.1 we simply requesting through the wiki url and apply the beautifulsoup for response format<p>

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url).text
soup = BeautifulSoup(response)
# soup

<p>2.2 Rendering within special division(for required table division)</p>

In [3]:
table_data = soup.find('div', class_='mw-parser-output')
table = table_data.table.tbody
# table

In [4]:
# find length of the total rows
rows_data=[ind for row in table.find_all('tr') for ind in row.find_all('td')]
len(rows_data)

180

<p>2.3 Set dictionary and rendering every tuple of the table</p>

In [5]:
columns = ['PostalCode', 'Borough', 'Neighbourhood']
# build dictionary for keys are the columns name
data = dict({key:[] for key in columns})

# selecting the element text
for row in table.find_all('tr'):
    for ind in row.find_all('td'):
        data['PostalCode'].append(ind.b.text)
        if ind.span.i:
          data['Borough'].append(ind.span.i.text)
          data['Neighbourhood'].append(ind.span.i.text)
        else:
          try:
              link_slect=ind.span.find_all('a')
              data['Borough'].append(link_slect[0].text)
              data['Neighbourhood'].append([rows.text for rows in link_slect[1:]])
          except:
              data['Borough'].append('Not assigned')
              data['Neighbourhood'].append('Not assigned')

print(data)



{'PostalCode': ['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B', 'M2B', 'M3B', 'M4B', 'M5B', 'M6B', 'M7B', 'M8B', 'M9B', 'M1C', 'M2C', 'M3C', 'M4C', 'M5C', 'M6C', 'M7C', 'M8C', 'M9C', 'M1E', 'M2E', 'M3E', 'M4E', 'M5E', 'M6E', 'M7E', 'M8E', 'M9E', 'M1G', 'M2G', 'M3G', 'M4G', 'M5G', 'M6G', 'M7G', 'M8G', 'M9G', 'M1H', 'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M7H', 'M8H', 'M9H', 'M1J', 'M2J', 'M3J', 'M4J', 'M5J', 'M6J', 'M7J', 'M8J', 'M9J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M7K', 'M8K', 'M9K', 'M1L', 'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M7L', 'M8L', 'M9L', 'M1M', 'M2M', 'M3M', 'M4M', 'M5M', 'M6M', 'M7M', 'M8M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M7N', 'M8N', 'M9N', 'M1P', 'M2P', 'M3P', 'M4P', 'M5P', 'M6P', 'M7P', 'M8P', 'M9P', 'M1R', 'M2R', 'M3R', 'M4R', 'M5R', 'M6R', 'M7R', 'M8R', 'M9R', 'M1S', 'M2S', 'M3S', 'M4S', 'M5S', 'M6S', 'M7S', 'M8S', 'M9S', 'M1T', 'M2T', 'M3T', 'M4T', 'M5T', 'M6T', 'M7T', 'M8T', 'M9T', 'M1V', 'M2V', 'M3V', 'M4V', 'M5V', 'M6V

<p>2.4 Dataframe column assign by the dictionary keys</p>

In [6]:
df = pd.DataFrame(columns = data.keys())
df

,PostalCode,Borough,Neighbourhood


<p>2.5 Put the dictionary data into dataframe with correspond keys as column</p>

In [7]:
# put the dictinoary data into dataframe based on the keys and columns name
for col_name in data.keys():
  df[col_name]=data[col_name]
print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"


<h3>3.Data preprocessing</h3>

<p>3.1 Remove the those rows which Borough is unknown</p>

In [8]:
filter_df=df[df['Borough']!='Not assigned']
new_df=filter_df.reset_index(drop=True)
print("New dataframe size is:{}".format(new_df.shape))
print("Total data we lost:{}".format((df.shape[0]-new_df.shape[0])))
new_df

New dataframe size is:(101, 3)
Total data we lost:79


,PostalCode,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Queen's Park,[]
5,M9A,Etobicoke,[Islington Avenue]
6,M1B,Scarborough,"[Malvern, Rouge]"
7,M3B,North York,[Don Mills]
8,M4B,East York,"[Parkview Hill, Woodbine Gardens]"
9,M5B,Downtown Toronto,"[Garden District, Ryerson]"


<p>3.2 Again,assign Borough as Neighborhood if Neighborhood is missig otherwise keep as it is.</p>

In [9]:
for i in range(len(new_df)):
    if new_df.loc[i,'Neighbourhood']==[]:
      new_df.loc[i,'Neighbourhood'].append(new_df.loc[i,'Borough'])
    else:
      new_df.loc[i,'Neighbourhood']=(new_df.loc[i,'Neighbourhood'])
new_df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Queen's Park,[Queen's Park]
5,M9A,Etobicoke,[Islington Avenue]
6,M1B,Scarborough,"[Malvern, Rouge]"
7,M3B,North York,[Don Mills]
8,M4B,East York,"[Parkview Hill, Woodbine Gardens]"
9,M5B,Downtown Toronto,"[Garden District, Ryerson]"


<p>3.3 Checking duplication of the PostalCode if it is then merged the neighbourhood of them</p>

In [10]:
unique_PostalCode=len(new_df['PostalCode'].unique())
unique_PostalCode==new_df.shape[0]

True

<i>it mean there is no duplication of the PostalCode in my dataframe so we left as it is</i>

<p>3.4 covert the list values into comma separated form of Neighborhood columns</p>

In [11]:
for i in range(new_df.shape[0]):
  new_df.loc[i,'Neighbourhood']=', '.join(new_df.loc[i,'Neighbourhood'])
print("Shape of the final datasets is:{}".format(new_df.shape))
new_df

Shape of the final datasets is:(101, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"
